In [1]:
# libraries
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# matplotlib defaults setup
plt.rcParams['savefig.dpi'] = 75
plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 14, 8
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "computer modern sans serif"

import numpy as np
from scipy.integrate import simps
import scipy.constants as cte
from scipy.sparse import diags

In [2]:
# grandezas de interesse em unidades atomicas
au_l = cte.value('atomic unit of length')
au_t = cte.value('atomic unit of time')
au_e = cte.value('atomic unit of energy')

# outras relacoes de interesse
ev = cte.value('electron volt')
au2ang = au_l / 1e-10
au2ev = au_e / ev

In [ ]:
# unidades do problema
E_0 = 150.0 # eV
L = 60.0 # angstron
sigma_x = 1.0 # angstron
x_0 = -20.0 # angstron
dt = dt_0 = 1e-15 # s

# unidades atomicas
E_0_au = E_0 / au2ev
L_au = L / au2ang
sigma_x_au = sigma_x / au2ang
x_0_au = x_0 / au2ang
dt_au = dt / au_t
k_0_au = np.sqrt(2 * E_0_au)

# salva os valores onde o algoritmo converge
par_convergentes = []

# divisor esperto (?)
de = lambda x: 2 if int((x/(10**(int(np.log10(x))-1)))%2) == 0 else 5

for N in [2**n for n in range(8,12)]:
    
    dt = dt_0
    
    # malha espacial
    x_au = np.linspace(-L_au/2, L_au/2, N)
    dx_au = x_au[1] - x_au[0]
    # diferencas finitas
    alpha = 1j / (2 * dx_au ** 2)
    beta = - 1j / (dx_au ** 2)
    diagonal_1 = [beta] * N
    diagonal_2 = [alpha] * (N - 1)
    diagonais = [diagonal_1, diagonal_2, diagonal_2]
    M = diags(diagonais, [0, -1, 1]).toarray()
    
    while True:
        #dt /= 10
        dt /= de(dt)
        dt_au = dt / au_t
        
        # pacote de onda
        PN = 1/(2*np.pi*sigma_x_au**2)**(1/4)
        psi = PN*np.exp(1j*k_0_au*x_au-(x_au-x_0_au)**2/(4*sigma_x_au**2))
        A0 = (simps(np.conjugate(psi)*psi,x_au)).real
        x_f_au = x_0_au 
        
        funcionou = True
        contador = 0
        norma = 100
        
        while x_f_au < -x_0_au:
            try:
                psi += dt_au * M.dot(psi)
                contador += 1
                if contador % 10 == 0:
                    A = (simps(np.conjugate(psi)*psi,x_au)).real
                    norma = 100 * A / A0
                    if np.abs(norma - 100) > 5:
                        funcionou = False
                        break
                    x_f_au = xm = (simps(np.conjugate(psi)* x_au * psi,x_au)).real / A
            except:
                funcionou = False
                break
                
        parametros = (N, dt, norma, contador)
        if funcionou:
            par_convergentes.append(parametros)
            break
        try:
            print("Estouro de norma: N = {}, dt={:.2e} s, norma = {:.2f}, contador = {}".format(*parametros))
        except:
            print("Grande estouro de norma: N = {}, dt={:.2e} s, contador = {}".format(N, dt, contador))
#                 xm2 = (simps(np.conjugate(psi)* x_au**2 * psi,x_au)).real / A
#                 sigma = np.sqrt(np.abs(xm2 - xm**2))
#                 print("A = {:.5f} %, <x> = {:.5f} A, sigma = {:.5f}".format(norma, xm, sigma))
#             if st % 10 == 0 or st == Ns-1:
#                 plt.plot(x_au, np.abs(psi))
#                 plt.show()
print(par_convergentes)

Estouro de norma: N = 256, dt=5.00e-16 s, norma = 4099223051973362430293387566720086060302336.00, contador = 10
Estouro de norma: N = 256, dt=1.00e-16 s, norma = 43728400528342117776434921472.00, contador = 10
Estouro de norma: N = 256, dt=5.00e-17 s, norma = 43997423325179333836800.00, contador = 10
Estouro de norma: N = 256, dt=1.00e-17 s, norma = 2322659270.13, contador = 10
Estouro de norma: N = 256, dt=5.00e-18 s, norma = 126837.07, contador = 10
Estouro de norma: N = 256, dt=1.00e-18 s, norma = 143.66, contador = 10
Estouro de norma: N = 256, dt=5.00e-19 s, norma = 109.54, contador = 10
Estouro de norma: N = 256, dt=1.00e-19 s, norma = 105.25, contador = 140
Estouro de norma: N = 256, dt=5.00e-20 s, norma = 105.06, contador = 540
Estouro de norma: N = 256, dt=1.00e-20 s, norma = 105.00, contador = 13350
Estouro de norma: N = 256, dt=5.00e-21 s, norma = 105.00, contador = 53370
Estouro de norma: N = 512, dt=5.00e-16 s, norma = 212047271075302424242987680578767193118867456.00, cont